In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from ),
    end_time=(fetch_data_to),
)

ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]


2025-03-03 23:21:58,309 INFO: Initializing external client
2025-03-03 23:21:58,310 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 23:21:59,321 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215664
Fetching data from 2025-02-03 04:21:58.307250+00:00 to 2025-03-04 03:21:58.307250+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.69s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-03 05:00:00+00:00,2,0
1,2025-02-03 06:00:00+00:00,2,0
2,2025-02-03 07:00:00+00:00,2,0
3,2025-02-03 08:00:00+00:00,2,0
4,2025-02-03 09:00:00+00:00,2,0
...,...,...,...
173687,2025-03-03 20:00:00+00:00,263,87
173688,2025-03-03 21:00:00+00:00,263,72
173689,2025-03-03 22:00:00+00:00,263,45
173690,2025-03-03 23:00:00+00:00,263,30


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173692 entries, 0 to 173691
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         173692 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  173692 non-null  int32                  
 2   rides               173692 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 2.7 MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173692 entries, 0 to 173691
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         173692 non-null  datetime64[us]
 1   pickup_location_id  173692 non-null  int32         
 2   rides               173692 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 2.7 MB


In [9]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [10]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,32,92,155,116,79,147,60,111,7,80,...,133,55,97,137,132,39,56,6,263,2025-02-21 10:00:00
1,2,1,0,0,0,0,0,0,1,0,...,0,0,25,5,0,0,0,0,255,2025-02-16 03:00:00
2,3,0,0,0,0,0,2,0,0,0,...,0,1,0,3,2,1,1,2,216,2025-02-03 08:00:00
3,0,0,0,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,147,2025-02-22 12:00:00
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,57,2025-02-20 06:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,1,0,0,1,0,2,1,9,4,0,...,2,1,1,3,1,0,0,1,256,2025-02-03 09:00:00
247,0,0,0,0,0,0,0,2,0,0,...,0,0,0,1,0,0,1,0,153,2025-02-18 13:00:00
248,0,0,0,0,0,0,0,1,6,2,...,0,0,2,0,2,4,1,0,225,2025-02-06 12:00:00
249,0,2,0,0,0,0,0,0,1,0,...,1,0,0,2,0,0,2,0,71,2025-02-13 18:00:00


In [11]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-03 23:22:09,750 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 23:22:09,754 INFO: Initializing external client
2025-03-03 23:22:09,754 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 23:22:11,108 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215664
Fetching data from 2025-02-03 04:22:09.750349+00:00 to 2025-03-04 03:22:09.750349+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (11.55s) 


In [12]:
current_date

Timestamp('2025-03-04 04:22:09.750349+0000', tz='Etc/UTC')

In [13]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-03 05:00:00
1,0,0,0,0,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,3,2025-03-03 05:00:00
2,0,2,3,3,4,6,2,2,2,4,...,1,1,3,1,0,1,2,0,4,2025-03-03 05:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-03 05:00:00
4,0,0,4,2,2,1,2,4,2,2,...,4,0,1,0,0,0,1,1,7,2025-03-03 05:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,2,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,259,2025-03-03 05:00:00
247,0,1,2,2,1,0,0,5,2,4,...,3,0,0,0,0,0,0,0,260,2025-03-03 05:00:00
248,2,2,5,11,10,7,26,13,25,37,...,10,7,5,0,0,0,0,2,261,2025-03-03 05:00:00
249,14,52,136,149,139,98,76,87,70,71,...,23,7,9,5,1,1,2,9,262,2025-03-03 05:00:00


In [14]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-03 23:22:28,601 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 23:22:28,603 INFO: Initializing external client
2025-03-03 23:22:28,603 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 23:22:29,486 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215664

In [15]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [16]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,1.0
3,6,0.0
4,7,1.0
...,...,...
246,259,0.0
247,260,0.0
248,261,1.0
249,262,14.0


In [17]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([132,  48, 186, 100, 230, 263, 162, 238, 141, 107], dtype=int32)